In [1]:
import pandas as pd
import numpy as np
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from Cleaner import Cleaner
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rahim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rahim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv(r"../data/news_dataset.csv")

In [3]:
X, y = df.text, df.category

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
pipeline = Pipeline(steps=[
    ("Cleaner", Cleaner()),
    ("CountVectorizer", CountVectorizer(stop_words=stopwords.words('english'))),
    ("RandomForestClassifier", RandomForestClassifier(n_estimators=1000)),
])

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('CountVectorizer',
                 CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=1000))])

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
confusion_matrix(y_test, y_pred)

array([[109,   0,   0,   0,   0],
       [  2,  72,   0,   2,   0],
       [  2,   1,  90,   0,   1],
       [  0,   0,   0,  81,   0],
       [  3,   1,   0,   1,  80]], dtype=int64)

In [9]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.94      1.00      0.97       109
entertainment       0.97      0.95      0.96        76
     politics       1.00      0.96      0.98        94
        sport       0.96      1.00      0.98        81
         tech       0.99      0.94      0.96        85

     accuracy                           0.97       445
    macro avg       0.97      0.97      0.97       445
 weighted avg       0.97      0.97      0.97       445



In [10]:
accuracy_score(y_test, y_pred)

0.9707865168539326